# Unified customer feedback POC

The idea is to use a single map-reduce pattern for multiple different sources of customer feedback:

- Reddit
- App stores
- (Maybe) formal reviews

We'll use the extract-organize-abstract pattern with map-reduce to handle extract-organize.

## Challenges

- The length of the input will vary drastically depending on the data source

## Steps

1. Init
1. Define the target
1. Fetch Reddit raw data
1. Fetch app review raw data
1. Format all into markdown
1. Pack
1. LLM pipeline

In [3]:
from core import Seed, init

init()

target = Seed.init("98point6", domain="98point6.com")

In [4]:
# The unified version that's been drastically refactored
import customer_experience
import general_search
import reddit.search

general_search_results = general_search.search_web(target)

app_store_urls = customer_experience.extract_app_store_urls(general_search_results)
reddit_urls = [
    result.link
    for result in reddit.search.find_submissions(
        target, num_results=5
    )
]
customer_experience_result = customer_experience.run(
    target, reddit_urls=reddit_urls, **app_store_urls
)

print(customer_experience_result.output_text)

2024-09-05 14:06:23.770 | DEBUG    | google_search:search:58 - Google search results: {'kind': 'customsearch#search', 'url': {'type': 'application/json', 'template': 'https://www.googleapis.com/customsearch/v1?q={searchTerms}&num={count?}&start={startIndex?}&lr={language?}&safe={safe?}&cx={cx?}&sort={sort?}&filter={filter?}&gl={gl?}&cr={cr?}&googlehost={googleHost?}&c2coff={disableCnTwTranslation?}&hq={hq?}&hl={hl?}&siteSearch={siteSearch?}&siteSearchFilter={siteSearchFilter?}&exactTerms={exactTerms?}&excludeTerms={excludeTerms?}&linkSite={linkSite?}&orTerms={orTerms?}&dateRestrict={dateRestrict?}&lowRange={lowRange?}&highRange={highRange?}&searchType={searchType}&fileType={fileType?}&rights={rights?}&imgSize={imgSize?}&imgType={imgType?}&imgColorType={imgColorType?}&imgDominantColor={imgDominantColor?}&alt=json'}, 'queries': {'request': [{'title': 'Google Custom Search - "98point6"', 'totalResults': '59900', 'searchTerms': '"98point6"', 'count': 10, 'startIndex': 1, 'language': 'lang_

# Positive Sentiment

## General Praise for the Company
- "Great experience! I highly recommend it to all the employees!" [(Artj3, Apple App Store, 2022-02-11)](https://apple/8344350122)
- "I am so thankful for having the option to use 98point6 for my care." [(Blakester!, Apple App Store, 2020-12-21)](https://apple/6778077902)
- "I love it. It was so easy to navigate and fast to respond." [(Jusnchoooo, Apple App Store, 2024-03-20)](https://apple/11065207384)
- "Great resource for banner employees! Highly recommend!!" [(teresagkt, Apple App Store, 2023-09-27)](https://apple/10413409475)
- "Awesome experience! Prompt care from the comfort of my own home." [(kdeaux22, Apple App Store, 2021-02-13)](https://apple/6989605440)
- "I had a very positive experience and had my questions answered quickly." [(suedreb, Apple App Store, 2022-10-27)](https://apple/9228928941)
- "I love this app. It was fast easy and perfect without having to wait for an appointment or spend big money on a co-pay for e

In [5]:
from core import make_experiment_dir

dir = make_experiment_dir(target)

with open(f"{dir}/customer_experience_baseline.md", "w") as f:
    f.write(customer_experience_result.output_text)

# Run 1: S6 / Palia

Overall the results were great

To improve:

- Citations are VERY inconsistent: (starry101, Source, 2024-04-09), (Mousekiwiiks, Source, 2023-08-17), (Anonymous, Steam, 2024-08-01), (Thumbs Up, Steam, 2024-08-21). I think this could be solved by making the citations better upstream
- It's heavily weighted towards Reddit even though Steam is probably a better data source. I'm guessing that it's simply that there's more Reddit text
- It took 3 minutes (!) to run

# Run 2: S6 / Palia

Changes:

1. Updated the citation format in both Reddit and Steam pipelines
2. Fetched 500 Steam reviews, up from 100
3. Trimmed a long document to 100k characters (it was 260k chars)

Results:

- Worked very nicely!
- One issue was multiple extracts from the same source, which made it look more serious
- 4 minutes :(

# Run 3: S6 / Palia

Changes:

1. URL shortener
2. Added the evaluation logger

It's a LOT faster, though it generated some malformed links like cache://... I'm also noticing that it tends to have more Steam content than the first run